## Data analysis
https://www.r-graph-gallery.com/101_Manhattan_plot.html

## Running sos notebook

On the Yale Farnam cluster,


On a local computer with for example 8 threads,

```
sos run BOLT-LMM_R_analysis_UKB.ipynb -q none -j 8
```


In [87]:
[global]
# Working directory
parameter: cwd = path('~/pleiotropy_UKB/data')
#Input individuals summary stats file
parameter: input_pattern = '*.snp_stats.gz'
import glob
input_file = sorted(glob.glob(input_pattern))
fail_if(len(input_file) == 0, msg = f"Input pattern ``{input_pattern}`` failed to match any files.")
#Output merged summary stats file
parameter: summary_stats_file = 'Test_INT-BMI.txt'

## Merging summary stats for all chromosomes

In [88]:
[step_1]
input: input_file
output: f'{cwd}/{summary_stats_file}.gz'
python: expand=True
import gzip
with gzip.open ('{_output}', 'wt') as outfile:
    with gzip.open({_input[0]:r}) as f:
        for line in f:
            outfile.write(line.decode('utf-8'))
        for files in [{_input:r,}][1:]:
            with gzip.open(files) as f:
                for line in f:
                    if not line.startswith(b'SNP'):
                        outfile.write(line.decode('utf-8'))

sum(1 for i in gzip.open('{_output}', 'r'))

## Creating the Manhattan plot

In [89]:
[step_2]
input: output_from('step_1')
output: f'{cwd}/{_input:bnn}.manhattan.png'
R: expand=True

    packages<-function(x){ x<-as.character(match.call()[[2]])
      if (!require(x,character.only=TRUE)){
        install.packages(pkgs=x,repos="http://cran.r-project.org")
        require(x,character.only=TRUE)
      }
    }
    packages(qqman)
    library(qqman)
    INTBMI_data <- read.table(gzfile({_input}), header=T)
    png({_output}, width = 6, height = 4, unit='in', res=300)
    bmi_manhattan <- manhattan(INTBMI_data, chr='CHR', bp='BP', snp='SNP', p='P_BOLT_LMM')
    dev.off()

Awk to find the minimum and maximun values in a specific column

In [ ]:
awk '(NR==1){Min=$3;Max=$3};(NR>=3){if(Min>$3) Min=$3;if(Max<$3) Max=$3} END {printf "The Min is %d ,Max is %d",Min,Max}' ukb_mfi_chr1_v3.txt

Filtering `.bgen` files using qctool2

```
qctool -g ukb_imp_chr1_v3.bgen -og subset_ukb_imp_chr1_v3.bgen -s ukb32285_imputedindiv.sample -incl-range 1:10177-20000 -incl-samples samples.txt
```

Then create a list of files to be combined using R. In this case the summary statistics for the association analysis and then create the dataset by binding files from chr{1:22}. The drawback is that R runs out of memory when doing this

In [ ]:
#install.packages("reader")
#intall.packages("plyr")
#install.packages("ggplot2")
#library(readr)
#library(plyr)
#library(ggplot2)
#file_list = Sys.glob("*.snp_stats.bgen.gz") #one way of creating lists
#file_list = list.files(path=mydir, pattern="*.snp_stats.bgen.gz", full.names=TRUE) #another way of creating lists
#data.list = lapply(file_list, function(x){read.table(file = x,header = TRUE, sep = "\t")})
#data.merged = do.call("rbind", data.list